# Modelación Híbrida de una Enfermedad Infecciosa

A continuación se implementará un **modelo computacional híbrido** que combina dos enfoques clave de modelación:  
la **Dinámica de Sistemas** (modelo SIR) y el **Modelado Basado en Agentes (ABM)**. El objetivo es simular la propagación de una enfermedad infecciosa en una población y comparar los resultados entre los tres enfoques: SIR puro, ABM puro y modelo híbrido.

---

## Dinámica de Sistemas - Modelo SIR

El modelo **SIR** es un modelo compartimental que representa a la población como grupos homogéneos. Divide a las personas en tres compartimentos:

- **S (Susceptibles):** Personas que pueden contraer la enfermedad.
- **I (Infectados):** Personas actualmente infectadas y que pueden contagiar.
- **R (Recuperados):** Personas que han superado la enfermedad y son inmunes.

Las transiciones entre estados se modelan con ecuaciones diferenciales:

* $ \frac{dS}{dt} = -\beta \cdot \frac{S \cdot I}{N} $

* $ \frac{dI}{dt} = \beta \cdot \frac{S \cdot I}{N} - \gamma \cdot I $

* $ \frac{dR}{dt} = \gamma \cdot I$

Donde:
- `β`: Tasa de transmisión (probabilidad de contagio por contacto).
- `γ`: Tasa de recuperación (inverso del tiempo promedio de enfermedad).
- `N`: Población total.

Este modelo es **determinista** y **agregado**: no considera diferencias individuales ni aleatoriedad.

---

## Modelado Basado en Agentes (ABM)

El modelo **ABM** representa cada persona como un **agente individual** con atributos y comportamientos propios:

- **Atributos:** Edad, cumplimiento de medidas sanitarias (como uso de mascarilla).
- **Estado:** Susceptible, Infectado o Recuperado.
- **Comportamiento:** Interacciones aleatorias con otros agentes.

La propagación ocurre de forma **probabilística**, dependiendo del cumplimiento individual y el número de contactos. Esto permite capturar fenómenos emergentes como:

- **Eventos de superpropagación.**
- **Diferencias por edad o comportamiento.**
- **Efectos estocásticos y variabilidad.**

Este modelo es más realista, pero también más complejo y menos predecible.

---

## Modelo Híbrido (SIR + ABM)

El **modelo híbrido** integra ambos enfoques para combinar lo mejor de cada uno:

1. **Inicialización:** El número de infectados iniciales proviene del modelo SIR.
2. **Simulación ABM:** Se modelan las interacciones y transmisiones individuales de forma probabilística.
3. **Retroalimentación:** Se calcula una **tasa de transmisión efectiva (β)** en cada paso basada en el comportamiento promedio de los agentes.
4. **Actualización SIR:** Se usan los valores de β e I para actualizar el modelo SIR de forma dinámica.

Esto permite simular:
- El impacto de políticas de salud pública sobre el comportamiento individual.
- Cómo el comportamiento individual afecta el curso general de la epidemia.
- Un equilibrio entre **simplificación matemática** y **realismo social**.

---
A continuación el código implementando los 3 enfoques para luego compararlos

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import random

In [8]:
# Parámetros globales
N = 500            # Tamaño de la población
I0 = 10            # Infectados iniciales
R0 = 0             # Recuperados iniciales
S0 = N - I0 - R0   # Susceptibles iniciales
beta_base = 0.3    # Tasa de transmisión base
gamma = 0.1        # Tasa de recuperación
dias = 160         # Días de simulación

### **Modelo SIR**

In [9]:
def modelo_sir(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

t = np.linspace(0, dias, dias)
sol_sir = odeint(modelo_sir, (S0, I0, R0), t, args=(N, beta_base, gamma))
S_sir, I_sir, R_sir = sol_sir.T

### **Modelo Basado en Agentes (ABM)**

In [10]:
class Agente:
    def __init__(self):
        self.estado = 'S'
        self.edad = random.randint(0, 100)
        self.cumplimiento = random.uniform(0, 1)
        self.dias_infectado = 0

# Inicializar agentes
agentes = [Agente() for _ in range(N)]
for i in random.sample(range(N), I0):
    agentes[i].estado = 'I'

hist_abm = {'S': [], 'I': [], 'R': []}

for _ in range(dias):
    nuevos_infectados = []
    nuevos_recuperados = []

    for i, agente in enumerate(agentes):
        if agente.estado == 'I':
            agente.dias_infectado += 1
            if random.random() < gamma:
                nuevos_recuperados.append(i)
        elif agente.estado == 'S':
            contactos = random.sample(agentes, 10)
            for otro in contactos:
                if otro.estado == 'I':
                    prob = beta_base * (1 - agente.cumplimiento)
                    if random.random() < prob:
                        nuevos_infectados.append(i)
                        break

    for i in nuevos_infectados:
        agentes[i].estado = 'I'
    for i in nuevos_recuperados:
        agentes[i].estado = 'R'

    S = sum(1 for a in agentes if a.estado == 'S')
    I = sum(1 for a in agentes if a.estado == 'I')
    R = sum(1 for a in agentes if a.estado == 'R')

    hist_abm['S'].append(S)
    hist_abm['I'].append(I)
    hist_abm['R'].append(R)

### **Modelo Híbrido**

In [11]:
# Inicializar de nuevo para el modelo híbrido
agentes_h = [Agente() for _ in range(N)]
for i in random.sample(range(N), I0):
    agentes_h[i].estado = 'I'

hist_hibrido = {'S': [], 'I': [], 'R': []}
S, I, R = S0, I0, R0

for dia in range(dias):
    # Calcular beta efectiva según cumplimiento promedio
    promedio_cumplimiento = np.mean([1 - a.cumplimiento for a in agentes_h if a.estado == 'S'])
    beta_efectiva = beta_base * promedio_cumplimiento

    # Paso SIR con beta dinámica
    SIR = odeint(modelo_sir, (S, I, R), [0, 1], args=(N, beta_efectiva, gamma))
    S, I, R = SIR[1]
    S = int(S)
    I = int(I)
    R = N - S - I  # Asegurar consistencia

    # Asignar nuevos estados a los agentes
    todos = list(range(N))
    random.shuffle(todos)
    for i in todos:
        agentes_h[i].estado = 'S'
    for i in todos[:I]:
        agentes_h[i].estado = 'I'
    for i in todos[I:I+R]:
        agentes_h[i].estado = 'R'

    hist_hibrido['S'].append(S)
    hist_hibrido['I'].append(I)
    hist_hibrido['R'].append(R)